# Sample script to automate testplans schedule

### Description

- This script enables users to automate the scheduling of test plans.
- It takes a list of test plan IDs and schedule them to the earliest available systems, considering availability up to the end of the current year.
- To schedule test plans across multiple fixtures, users can add a custom property named `Number of fixtures` to each test plan, specifying how many fixtures should be used for its execution. If this property is not provided or is greater than 8, it will default to 1.

Note: To customize the scheduling algorithm, modify the [Actions](#Actions) and the [APIs and algorithms](#APIs-and-algorithms)

## Imports

In [ ]:
import requests
import os
from datetime import datetime, timedelta, timezone
from typing import Dict, List, Tuple
import scrapbook as sb

from nisystemlink.clients.systems import SystemsClient
from nisystemlink.clients.systems.models import QuerySystemsRequest
from nisystemlink.clients.assetmanagement import AssetManagementClient
from nisystemlink.clients.assetmanagement.models import QueryAssetsRequest
from nisystemlink.clients.test_plan import TestPlanClient
from nisystemlink.clients.test_plan.models import QueryTestPlansRequest, ScheduleTestPlansRequest, ScheduleTestPlanRequest, OrderBy, TestPlan

## Parameters

- **`test_plan_ids`**: IDs of the test plans to be scheduled.

Parameters are also listed in the metadata for the **parameters cell**, along with their default values.  
The Notebook Execution services use that metadata to pass parameters to this notebook.  
To view the metadata:
- Select the code cell
- Click the **wrench icon** in the right panel.

### Sample metadata

```json
{
  "papermill": {
    "parameters": {
      "result_ids": []
    }
  },
  "systemlink": {
    "namespaces": [],
    "parameters": [
      {
        "display_name": "Test Plan IDs",
        "id": "test_plan_ids",
        "type": "string[]"
      }
    ],
    "version": 2
  },
  "tags": ["parameters"]
}
```
For more information on how parameterization works, review the [papermill documentation](https://papermill.readthedocs.io/en/latest/usage-parameterize.html#how-parameters-work).

In [ ]:
test_plan_ids = [""] #Add the test plan ids as list of strings here

## Constants and objects

In [ ]:
MAX_SYSTEMS_COUNT = 1000 # Max systems to be queried
MAX_FIXTURES_COUNT = 1000 # Max fixtures to be queried
MAX_TESTPLANS_COUNT = 10000 # Max testplans to be queried
MAX_TESTPLANS_COUNT_PER_QUERY = 1000 # Max testplans to be queried per query
DEFAULT_TESTPLAN_DURATION = 86400 # Default estimated duration as 1 day

current_time = datetime.now(timezone.utc)
start_time = current_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
end_of_year = datetime(current_time.year, 12, 31, 23, 59, 59, tzinfo=timezone.utc)
due_date_time = end_of_year.strftime("%Y-%m-%dT%H:%M:%S.000Z")

system_client = SystemsClient()
asset_client = AssetManagementClient()
test_plan_client = TestPlanClient()

## APIs and algorithms

In [ ]:
# get testplan to schedule

def fetch_testplan_and_system_filter(testplan_to_schedule_id):
    testplan_data = test_plan_client.get_test_plan(testplan_to_schedule_id)
    if testplan_data is not None:
        number_of_fixtures = testplan_data.properties["Number of fixtures"]
        try:
            number_of_fixtures = int(number_of_fixtures)
        except (ValueError, TypeError):
            number_of_fixtures = 1

        return testplan_data, number_of_fixtures
    else:
        return None, None

In [ ]:
# get systems

def query_systems(system_filter):
    system_request_payload = QuerySystemsRequest(
        take = MAX_SYSTEMS_COUNT,
        projection = "new(id,alias)",
        orderBy = "lastUpdatedTimeStamp DESC",
        filter = system_filter
    )

    query_response = system_client.query_systems(system_request_payload)
    return query_response.data

In [ ]:
# get fixtures

def query_fixtures(systems):
    ids_string  = ",".join(f'"{item["id"]}"' for item in systems)

    fixture_request_payload = QueryAssetsRequest(
        take = MAX_FIXTURES_COUNT,
        filter = f'AssetType == "FIXTURE" && (locations.Any(l => new[]{{{ids_string}}}.Contains(l.minionId)))',
        orderBy = "LAST_UPDATED_TIMESTAMP",
        descending = True
    )

    query_response = asset_client.query_assets(fixture_request_payload)

    return query_response.assets

In [ ]:
# convert fixtures and systems to dict (systemId as key and list of fixture ids as value)

def get_system_fixtures_map(fixtures):
    result = {}
    for item in fixtures:
        obj_key = item.location.minion_id
        item_id = item.id
        
        if obj_key not in result:
            result[obj_key] = []
        
        result[obj_key].append(item_id)
    return result

In [ ]:
# get testplans for systems

def query_testplans(systems):
    n = len(systems)
    condition_strings = [f'systemId == "{system["id"]}"' for system in systems]
    
    final_string = ' or '.join(condition_strings)
    testplans_filter_string = f'({final_string}) and (plannedStartDateTime <= "{due_date_time}" and estimatedEndDateTime >= "{start_time}")'

    testplans = []
    continuation_token = None

    while len(testplans) < MAX_TESTPLANS_COUNT:
        query_testplans_payload = QueryTestPlansRequest(
            filter = testplans_filter_string,
            take = MAX_TESTPLANS_COUNT_PER_QUERY,
            descending = True,
        )

        if continuation_token:
            query_testplans_payload.continuation_token = continuation_token

        testplans_data = test_plan_client.query_test_plans(query_request=query_testplans_payload)
        continuation_token = testplans_data.continuation_token

        if testplans_data.test_plans is not None: 
            testplans.extend(testplans_data.test_plans)

            if len(testplans_data.test_plans) < MAX_TESTPLANS_COUNT_PER_QUERY or continuation_token is None:
                break
        else:
            break

    return testplans[:MAX_TESTPLANS_COUNT]

In [ ]:
#convert testplans and systems to dict (systemId as key and testplan's planned start and end time as value )

def organize_testplans_by_system(testplans, systems):
    system_with_testplans = {}

    # Populate the dictionary with test plans
    for testplan in testplans:
        system_id = testplan.system_id
        testplan_info = {
            'start_time': testplan.planned_start_date_time,
            'end_time': testplan.estimated_end_date_time
        }
        if system_id not in system_with_testplans:
            system_with_testplans[system_id] = []
        system_with_testplans[system_id].append(testplan_info)

    # Ensure all systems are represented in the dictionary
    for system in systems:
        system_id = system['id']
        if system_id not in system_with_testplans:
            system_with_testplans[system_id] = []

    return system_with_testplans

In [ ]:
#sort testplans in system based on testplan start time

def sort_and_filter_system_testplans(system_testplans: Dict[str, List[Dict[str, str]]]) -> Dict[str, List[Dict[str, str]]]:
    for system_id, testplans in system_testplans.items():
        testplans.sort(key=lambda x: datetime.fromisoformat(x['end_time'].replace("Z", "+00:00")))

        testplans = [testplan for testplan in testplans if datetime.fromisoformat(testplan['end_time'].replace("Z", "+00:00")) > current_time]

        system_with_testplans[system_id] = testplans

    return system_with_testplans

In [ ]:
#find available time slots in all systems to schedule, sort it based on time and return the first slot.

def find_available_slots(system_with_testplans: Dict[str, List[Dict[str, str]]], test_plan: TestPlan) -> List[Dict[str, str]]:
    duration = timedelta(seconds=test_plan.estimated_duration_in_seconds if test_plan.estimated_duration_in_seconds != None else 86000)

    available_slots = []

    for system_id, occupied_slots in system_with_testplans.items():
        occupied_slots.sort(key=lambda x: datetime.fromisoformat(x['start_time'].replace("Z", "+00:00")))
        
        last_end_time = current_time

        for slot in occupied_slots:
            start_time = datetime.fromisoformat(slot['start_time'].replace("Z", "+00:00"))
            end_time = datetime.fromisoformat(slot['end_time'].replace("Z", "+00:00"))

            if start_time > last_end_time + duration:
                available_slots.append({
                    'system_id': system_id,
                    'planned_start_time': last_end_time.isoformat(),
                    'estimated_end_time': (last_end_time + duration).isoformat()
                })
                break

            last_end_time = max(last_end_time, end_time)

        if last_end_time + duration < datetime.fromisoformat(due_date_time.replace("Z", "+00:00")):
            available_slots.append({
                'system_id': system_id,
                'planned_start_time': last_end_time.isoformat(),
                'estimated_end_time': (last_end_time + duration).isoformat()
            })

    available_slots.sort(key=lambda x: datetime.fromisoformat(x['planned_start_time']))

    return available_slots

In [ ]:
#schedule api call

def schedule_testplan(testplan_to_schedule_id, available_slot, testplan_to_schedule, fixtureIds):
    schedule_test_plans_request = ScheduleTestPlansRequest(
        test_plans=[
            ScheduleTestPlanRequest(
                id = testplan_to_schedule_id,
                planned_start_date_time= available_slot['planned_start_time'],
                estimated_end_date_time= available_slot['estimated_end_time'],
                system_id = available_slot['system_id'],
                fixture_ids = available_slot['fixtures'],
                estimated_duration_in_seconds = testplan_to_schedule.estimated_duration_in_seconds
            )
        ],
        replace = True
    )

    schedule_test_plans_response = test_plan_client.schedule_test_plans(schedule_test_plans_request)

    if schedule_test_plans_response.failed_test_plans is None:
        return True
    else:
        return False

In [ ]:
# get available slot considering system and fixtures

def get_available_slot(available_slots, system_fixtures_map, number_of_fixtures_to_select):
    if(number_of_fixtures_to_select == 0):
        return available_slots[0] if available_slots else None
    
    for slot in available_slots:
        system_id = slot['system_id']
        fixture_ids = system_fixtures_map.get(system_id, [])
        
        if len(fixture_ids) >= number_of_fixtures_to_select:
            selected_fixture_ids = fixture_ids[:number_of_fixtures_to_select]
            slot['fixture_ids'] = selected_fixture_ids
            return slot
    
    return available_slots[0] if available_slots else None

In [ ]:
def merge_intervals(intervals: List[Tuple[datetime, datetime]]) -> List[Tuple[datetime, datetime]]:
    if not intervals:
        return []
    intervals.sort()
    merged = [intervals[0]]
    for current_start, current_end in intervals[1:]:
        last_start, last_end = merged[-1]
        if current_start <= last_end:
            merged[-1] = (last_start, max(last_end, current_end))
        else:
            merged.append((current_start, current_end))
    return merged

In [ ]:
def invert_intervals(intervals: List[Tuple[datetime, datetime]], start: datetime, end: datetime) -> List[Tuple[datetime, datetime]]:
    available = []
    prev_end = start
    for busy_start, busy_end in intervals:
        if prev_end < busy_start:
            available.append((prev_end, busy_start))
        prev_end = max(prev_end, busy_end)
    if prev_end < end:
        available.append((prev_end, end))
    return available

In [ ]:
def find_earliest_common_slot(fixtures: List[str], system: str, testPlans, 
                               n_required: int, duration: timedelta,
                               search_start: datetime, search_end: datetime):
    system_busy = []
    fixture_busy = {slave: [] for slave in fixtures}

    for testPlan in testPlans:
        planned_start_time = testPlan.planned_start_date_time if isinstance(testPlan.planned_start_date_time, datetime) else datetime.fromisoformat(testPlan.planned_start_date_time)
        estimated_end_time = testPlan.estimated_end_date_time if isinstance(testPlan.estimated_end_date_time, datetime) else datetime.fromisoformat(testPlan.estimated_end_date_time)

        if testPlan.system_id == system and len(testPlan.fixture_ids) == 0:
            system_busy.append((planned_start_time, estimated_end_time))
        for fixture in testPlan.fixture_ids:
            if fixture in fixture_busy:
                fixture_busy[fixture].append((planned_start_time, estimated_end_time))

    system_busy = merge_intervals(system_busy)

    for fixture in fixtures:
        combined = fixture_busy[fixture] + system_busy
        fixture_busy[fixture] = merge_intervals(combined)

    fixture_available = {}
    for fixture in fixtures:
        fixture_available[fixture] = invert_intervals(fixture_busy[fixture], search_start, search_end)

    time_points = set()
    for intervals in fixture_available.values():
        for start, end in intervals:
            time_points.add(start)
            time_points.add(end)
    time_points = sorted(time_points)

    for time_point in time_points:
        available_fixtures = []
        for fixture, intervals in fixture_available.items():
            for interval_start, interval_end in intervals:
                if interval_start <= time_point and interval_end >= time_point + duration:
                    available_fixtures.append(fixture)
                    break

        if len(available_fixtures) >= n_required:
            return {
                'planned_start_time': time_point,
                'estimated_end_time': time_point + duration,
                'fixtures': available_fixtures[:n_required]
            }
    return None

In [ ]:
def find_best_system(system_fixture_map: Dict[str, List[str]], testPlans, 
                     n_required: int, duration: timedelta,
                     search_start: datetime, search_end: datetime):
    best_option = None

    for system, fixtures in system_fixture_map.items():
        result = find_earliest_common_slot(fixtures, system, testPlans, n_required, duration, search_start, search_end)
        if result:
            if not best_option or result['planned_start_time'] < best_option['planned_start_time']:
                best_option = {
                    'system_id': system,
                    'planned_start_time': result['planned_start_time'],
                    'estimated_end_time': result['estimated_end_time'],
                    'fixtures': result['fixtures']
                }

    return best_option

## Actions

In [ ]:
scheduled_testplans = []
scheduled_testplan_names = []
unScheduled_testplans = []

# Loop through each testplan_id in the list
for testplan_to_schedule_id in test_plan_ids:
    # Fetch the test plan and system filter
    testplan_to_schedule, number_of_fixtures_to_schedule = fetch_testplan_and_system_filter(testplan_to_schedule_id)

    if testplan_to_schedule is not None:
        # Get systems based on the system filter
        systems = query_systems(testplan_to_schedule.system_filter)

        if systems is not None:
            # Get Fixtures based on systems
            fixtures = query_fixtures(systems)
            system_fixtures_map = get_system_fixtures_map(fixtures)

            # Get testplans for the identified systems
            testplans = query_testplans(systems)

            is_scheduled = False

            duration = timedelta(seconds=testplan_to_schedule.estimated_duration_in_seconds if testplan_to_schedule.estimated_duration_in_seconds != None else 86000)
            best_system = find_best_system(system_fixtures_map, testplans, number_of_fixtures_to_schedule, duration, current_time, datetime.fromisoformat(due_date_time.replace("Z", "+00:00")))

            if(best_system):
                fixtureIds = best_system['fixtures']

                best_system['planned_start_time'] =  best_system['planned_start_time'].isoformat()
                best_system['estimated_end_time'] = best_system['estimated_end_time'].isoformat()
                
                is_scheduled = schedule_testplan(testplan_to_schedule_id, best_system, testplan_to_schedule, fixtureIds)
            else:
                # Organize the test plans by system (systemId as key and testplans as value)
                system_with_testplans = organize_testplans_by_system(testplans, systems)
    
                # Sort test plans in each system based on start time
                sorted_system_testplans = sort_and_filter_system_testplans(system_with_testplans)
    
                # Find available slots for scheduling the test plan
                available_slots = find_available_slots(sorted_system_testplans, testplan_to_schedule)
    
                available_slot = get_available_slot(available_slots, system_fixtures_map, number_of_fixtures_to_schedule)
                fixtureIds = system_fixtures_map.get(available_slot['system_id'], [])
                if(len(fixtureIds) > number_of_fixtures_to_schedule):
                    fixtureIds = fixtureIds[:number_of_fixtures_to_schedule]

                is_scheduled = schedule_testplan(testplan_to_schedule_id, available_slot, testplan_to_schedule, fixtureIds)

            if testplan_to_schedule is not None and is_scheduled:
                scheduled_testplan_names.append(testplan_to_schedule.name)
                scheduled_testplans.append(testplan_to_schedule_id)
            else:
                unScheduled_testplans.append(testplan_to_schedule_id)
        else:
            unScheduled_testplans.append(testplan_to_schedule_id)

    else:
        unScheduled_testplans.append(testplan_to_schedule_id)     

## Executions page output

In [ ]:
test_plan_names = ', '.join(scheduled_testplan_names)
sb.glue("Total test plans", len(test_plan_ids))
sb.glue("Scheduled test plans", test_plan_names)

if len(unScheduled_testplans) == 0:
    sb.glue("Unscheduled test plans",  "-")
else:
    sb.glue("Unscheduled test plans",  unScheduled_testplans)

## Output

In [ ]:
print("Total test plans:", len(test_plan_ids))
print("Scheduled test plans:", test_plan_names)
if len(unScheduled_testplans) == 0:
    print("Unscheduled test plans: -")
else:
    print("Unscheduled test plans:", unScheduled_testplans)

## Next Steps
- Publish this notebook to SystemLink by right-clicking it in the JupyterLab File Browser with the interface as Test Plan Scheduler.